In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import chi2

/home/a20114261/anaconda3/envs/GeneInteractionsBN/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [7]:
#Initial data setup

X = pd.read_csv("/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Batches/censored_Erk_Target/batch_0.csv")

# each cell can be -1, 0 or 1
#Pre-processing data
X['Pvalue']=list(map(lambda x: 1 if x<=0.01 else (0 if x>=0.5 else -1), X['Pvalue']))
del X['Unnamed: 0']
del X['TrueIndex']
del X['CauseGene']
del X['EffectGene']
del X['Replicate']
del X['Treatment']
PC=[]
for column in X:
    if (column != "Pvalue"):
        X[column] = list(map(lambda x: x+1 , X[column]))
data_c_size  = len(X['Pvalue'])
data_vars = [0,1,2,3]

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0,1,1,0,2,1,2,2,1,2,...,0,1,1,0,1,1,1,1,1,1
1,2,2,1,2,2,1,2,2,1,0,...,0,0,1,0,0,0,0,0,1,0
2,2,2,1,2,2,1,2,2,1,0,...,0,0,1,1,1,1,1,1,1,1
3,2,2,1,2,2,1,2,2,1,2,...,0,0,1,1,1,1,1,1,1,1
4,2,2,1,2,2,1,2,2,1,0,...,0,0,1,0,0,1,0,0,1,0


In [163]:
X = pd.read_csv("./Alarm1_s1000_v1.txt",delimiter='  ',header=None)
x_heads = []
for i in range(0,len(X.keys())):
    max_v=1
    for row in X[X.keys()[i]]:
        if (row>max_v-1):
            max_v+=row-(max_v-1)
    save_i=i+1
    x_heads.append("Node"+str(save_i)+"@"+str(max_v))
X.columns = x_heads
data_c_size = len(X[X.keys()[1]])
data_vars = [0,1,2,3]

/home/a20114261/anaconda3/envs/GeneInteractionsBN/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [165]:
X.head()

,Node1@4,Node2@4,Node3@3,Node4@4,Node5@3,Node6@2,Node7@3,Node8@3,Node9@2,Node10@3,...,Node28@2,Node29@3,Node30@2,Node31@2,Node32@3,Node33@3,Node34@3,Node35@3,Node36@2,Node37@2
0,0,1,1,0,2,1,2,2,1,2,...,0,1,1,0,1,1,1,1,1,1
1,2,2,1,2,2,1,2,2,1,0,...,0,0,1,0,0,0,0,0,1,0
2,2,2,1,2,2,1,2,2,1,0,...,0,0,1,1,1,1,1,1,1,1
3,2,2,1,2,2,1,2,2,1,2,...,0,0,1,1,1,1,1,1,1,1
4,2,2,1,2,2,1,2,2,1,0,...,0,0,1,0,0,1,0,0,1,0


In [146]:
# Initial setup for TensorFlow
with tf.device('/job:localhost/replica:0/task:0/device:GPU:1'):
    a = tf.placeholder(tf.float32,shape=(data_c_size,))
    b = tf.placeholder(tf.float32,shape=(data_c_size,))
    dot_mult_op = tf.einsum('i,i->',a, b)
    
    c = tf.placeholder(tf.float32,shape=(data_c_size,))
    d = tf.placeholder(tf.float32,shape=(data_c_size,))
    var_mask = tf.equal(c,d)
    
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.10)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
                 


In [7]:
def tf_const_vector(n,size):
    global sess
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:1'):
        a = tf.constant(np.zeros(size),tf.float32)
        b = tf.constant(n,tf.float32)
        v_constructor = a+b
        return sess.run(v_constructor)

In [8]:
def tf_dot_mult(arr1,arr2):
    global sess
    global dot_mult_op
    return sess.run(dot_mult_op,{a:arr1,b:arr2})

In [9]:
def tf_mask_var(arr1,var):
    global sess
    global var_mask
    global const_vectors
    return sess.run(var_mask,{c:arr1,d:const_vectors[var]})

In [22]:
def tf_mask_equal(arr1,arr2):
    global sess
    global var_mask
    return sess.run(var_mask,{c:arr1,d:arr2})

In [23]:
def setup_const_vectors(var_arr,size):
    returnable_dict = {}
    for i in range(len(var_arr)):
        returnable_dict[var_arr[i]] = tf_const_vector(var_arr[i],size)
    
    return returnable_dict

In [11]:
def ExistDseparator(TargetNode,Xi, Z, X, alpha):
    flagExist = False
    dsepSet=[]
    #counter=0
    for i in range(0,(2**len(Z))-1):
        IDsubsetZ_dec = i
        IDsubsetZ_bin = bin(IDsubsetZ_dec)
        subsetZ = getZsubset(IDsubsetZ_bin,Z)
        # no cache
        #print("from exist dseparator")
        dep = Dep(TargetNode,Xi,subsetZ, X, alpha)
        #counter+=1
        if (dep==0):
            flagExist = True
            dsepSet = subsetZ
            break
    #print("Module exist d-separator: ",counter)
    return [flagExist,dsepSet]

In [12]:
def GetCondValues(z, condVars):
    if len(condVars)>1:
        maxz = 1
        for column in condVars:
            maxz*=int(column['name'].split('@')[1])
        d = np.zeros((1,len(condVars)))
        
        div_dim = maxz/int(condVars[0]['name'].split('@')[1])
        num2div = z-1
        
        for i in range(0,len(condVars)-1):
            d[i] = floor(num2div/div_dim)
            num2div=mod(num2div,div_dim)
            div_dim = div_dim/condVars[i+1]
        d[i+1] = num2div
    else:
        d = [z]
    return d

In [190]:
def Dep(TargetNode, Xi, CondVars, X, alpha):
    #print('in Dep')
    global data_c_size
    
    vvTarget = []
    for i in range(int(TargetNode['name'].split('@')[1])):
        vvTarget.append(i)
    
    vvXi = []
    for i in range(int(Xi['name'].split('@')[1])):
        vvXi.append(i)
    
    szCondVars = 1
    #print(CondVars)
    for column in CondVars:
        #print(column)
        szCondVars *= int(column['name'].split('@')[1])
    
    if data_c_size*3 <= (5 * len(vvTarget)*len(vvXi)*(szCondVars)):
        return 1
    
    if (len(CondVars)==0):
        S = np.zeros((len(vvTarget),len(vvXi)))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                var1=np.array(TargetNode['data']) # Target Node
                var2=np.array(Xi['data'])
                # looking up for 0, 1 and 2
                op1 = tf_mask_var(var1,vvTarget[i])
                #op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                op2 = tf_mask_var(var2,vvXi[j])
                #op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                S[i][j] = tf_dot_mult(op1,op2)
                #S[i][j]=np.sum(op1*op2)
                #print(S[i][j])
        G2=0
        N = np.sum(S)
        print(S)
        print(N)
        Si =np.sum(S,axis=1)
        Sj =np.sum(S,axis=0)
        Df = ((len(vvTarget)-1)*(len(vvXi)-1))
        print(S[np.where(S>0)])
        Dedf = len((S[np.where(S>0)]))\
                      - len(Si[np.where(Si>0)])\
                      - len(Sj[np.where(Sj>0)]) +1
        print("Dedf before if: ",Dedf)
        if (Dedf<1):
            Dedf=1
            #print('Here is the trouble')
            #return 0
        #print(S)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                #print(i)
                #print(j)
                if (S[i][j]>0):
                    #print(Si[i])
                    #print(Sj[j])
                    G2 = G2 + S[i][j]*np.log((S[i][j])*N/(Si[i]*Sj[j]))
                    #print(G2)
        G2 = 2*G2
        
    else: # test conditional dependency
        
            
        S = np.zeros((len(vvTarget),len(vvXi),szCondVars))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    condValue = GetCondValues(k,CondVars)
                    #print(condValue)
                    flagDataCondVars = np.ones((len(vvXi),1))
                    for l in range(0,len(CondVars)):
                        X_l = CondVars[l]
                        #flagDataCondVars = flagDataCondVars*(X_l[where(X_l==condValue[l])])
                        op1 = tf_mask_var(X_l['data'],condValue[l])
                        #flagDataCondVars = np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l.tolist())))
                        flagDataCondVars = flagDataCondVars*op1
                    var1=np.array(TargetNode['data'])
                    var2=np.array(Xi['data'])
                    # looking up for -1, 0 and 1
                    #op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                    op1 = tf_mask_var(var1,vvTarget[i])
                    #print(op1)
                    op2 = tf_mask_var(var2,vvXi[j])
                    #op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                    #print(op2)
                    #print(op1*op2*flagDataCondVars)
                    S[i][j][k]=np.sum(op1*op2*flagDataCondVars)
        G2 = 0
        
        Sjk = np.sum(S,axis=0)
        Sik = np.sum(S,axis=1)
        Sk = np.sum(Sjk,axis=0)
        
        #Sjk = sum(S,1);
        #Sik = sum(S,2);
        #Sk = sum(Sjk,2); 
        Dedf = len(S[np.where(S>0)]) -\
                len(Sik[np.where(Sik>0)]) -\
                len(Sjk[np.where(Sjk>0)]) +\
                len(Sk[np.where(Sk>0)])
        if Dedf>1:
            Dedf=1
            #print('No, here is the trouble')
            #return 0
        
        # compute G2 statistic
        #print(S.shape)
        #print(Sjk.shape)
        #print(Sik.shape)
        #print(Sk.shape)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    if (S[i][j][k]>0):
                        #print(i,j,k)
                        #test_op =  Sk[k]
                        #test_op = Sik[i][k]
                        #test_op = Sjk[j][k]
                        G2=G2+(S[i][j][k] * np.log(\
                                    (S[i][j][k]) * Sk[k])/\
                                    (Sik[i][k] * Sjk[j][k]))
        G2 = 2*G2
    
    assoc = (alpha - (1 - chi2.cdf(G2,Dedf)))/alpha
    #print(G2)
    #print(Dedf)
    #print(assoc)
    if assoc<0:
        assoc=0
    print("Target Node: ",TargetNode['name'])
    print("Node XI: ", Xi['name'])
    print("\tG2:",G2)
    print("\tDedf: ",Dedf)
    print("\tAssoc: ",assoc)
    print()
    print()
    return assoc

In [14]:
def getZsubset(bin_id,Z):
    bin_str=str(bin_id)
    Zsubset=[]
    for i in range(0,len(bin_str)-2):
        if bin_str[i]==1:
            Zsubset.append(Z[i])
    return Zsubset

In [128]:
def arrayUniverse(TargetNode,X):
    Universe = []
    for key in X:
        #print("Key in X:",key)
        #print("Target Node is: ",TargetNode)
        if (key!=TargetNode):
            append_dict={'name':key,'data':X[key].copy(deep=True).tolist()}
            Universe.append(append_dict)
    return Universe

In [16]:
def arrayX(X):
    returnable=[]
    for key in X:
        #tagged_c = MB_Column(X[key].copy(deep=True).tolist(),key)
        append_dict={'name':key,'data':X[key].copy(deep=True).tolist()}
        returnable.append(append_dict)
    return returnable

In [67]:
def MinAssoc(TargetNode, Xi,Z, fixedCondVars, X, alpha):
    
    min_assoc=999
    #counter=0
    #print(fixedCondVars)
    if len(Z)==0:
        #print("from minassoc1")
        #print(fixedCondVars)
        min_assoc = Dep(TargetNode, Xi, fixedCondVars, X, alpha)
        #counter+=1
        subsetZ_min_assoc = fixedCondVars
        #print(min_assoc)
    else:
        print(2**len(Z)-1)
        for IDsubsetZ_dec in range(0,2**len(Z)-1):
            IDsubsetZ_bin = bin(IDsubsetZ_dec)
            subsetZ = getZsubset(IDsubsetZ_bin,Z)
            #print(len(Xi))
            #print("from minassoc2")
            subsetZ_assoc=Dep(TargetNode, Xi, fixedCondVars+subsetZ,X,alpha)
            #counter+=1
            #print(subsetZ_assoc[IDsubsetZ_dec])
            if subsetZ_assoc < min_assoc:
                min_assoc = subsetZ_assoc
                subsetZ_min_assoc = fixedCondVars + subsetZ
                if (min_assoc==0):
                    break
    #print("Min Assoc module: ",counter)
    return min_assoc

In [138]:
def MaxMinHeuristic(TargetNode, CPC, Universe, X, alpha):
    F=[]
    assocF=-1
    Z = CPC
    fixedCondVars = []
    if (len(CPC)>0):
        Z = CPC[0:-1]           # all but the last one   
        fixedCondVars = [CPC[-1]] # we use last one
    
    #for i in range(len(Universe)-1,0,-1):
    for i in range(0,len(Universe)):
        if (len(Universe[i])==0):
            continue
        minAssoc = MinAssoc(TargetNode,Universe[i], Z,fixedCondVars, X,alpha)
        #print(minAssoc)
        if minAssoc>assocF:
            assocF = minAssoc
            F=Universe[i]
        if minAssoc == 0:
            Universe[i]=[]
            #Universe.pop(i)
        #print(F)
    return [F,assocF,Universe]

In [51]:
def MMPC_(TargetNode,Universe,X,alpha):
    
    CPC=[]
    #print(TargetNode)
    # Phase I: Foward
    print("Entering Phase I")
    while len(Universe)>0:
        CPC_old = list(CPC) # copy
        maxminheur=MaxMinHeuristic(TargetNode,CPC,list(Universe),X,alpha)
        # maxminheur = [F, assocF, Universe]
        F = maxminheur[0]
        assocF = maxminheur[1]
        Universe = maxminheur[2]
        #print("Universe printing line")
        #print(Universe)
        print("=============================")
        #print(CPC)
        if assocF > 0:
            CPC.append(F)
            indF=Universe.index(F)
            len(Universe)
            Universe.pop(indF)
            len(Universe)
        if (len(CPC)==len(CPC_old)):
            break
        print("Universe actual size:")
        print(len(Universe))
        print("CPC actual size:")
        print(len(CPC))
    
    # Phase 2: Backward
    print("Entering Phase II")
    if len(CPC)>1:
        print("CPC size was larger than 1")
        for i in range(0,len(CPC)):
            Z=list(CPC)
            # index is i
            Z[i]=[]
            if ExistDseparator(TargetNode,CPC[i],Z,X,alpha)[0] == True:
                CPC[i]=[]
    return CPC

In [20]:
def MMPC(TargetNode, X, alpha):
    # The universe will be an array of DataFrame Columns
    Universe = arrayUniverse(TargetNode,X)
    print(len(Universe))
    X = arrayX(X)
    print(len(X))
    for column in X:
        if (column['name']==TargetNode):
            TargetNode = column
            break
    CPC = MMPC_(TargetNode,Universe,X,alpha)
    return CPC

In [191]:
alpha=0.05
# Initial vectors setup
const_vectors = setup_const_vectors(data_vars,data_c_size)
#for i in range(0,len(X.keys())):    
#for i in range(0,1):    
print("======================================================================================")
TargetNode =X.keys()[0]
PC=MMPC(TargetNode,X,alpha)
print("Target Node is: ",TargetNode)
print("PC set is :")
for i in PC:
    print(column['name'])

36
37
Entering Phase I
[[  64.   28.   12.   12.]
 [   2.   24.   17.   18.]
 [   7.    9.  734.   36.]
 [   1.    2.   15.   19.]]
1000.0
[  64.   28.   12.   12.    2.   24.   17.   18.    7.    9.  734.   36.
    1.    2.   15.   19.]
Dedf before if:  9
Target Node:  Node1@4
Node XI:  Node2@4
	G2: 592.571218428
	Dedf:  9
	Assoc:  1.0


[[   4.  101.   11.]
 [   5.   53.    3.]
 [  29.  716.   41.]
 [   3.   34.    0.]]
1000.0
[   4.  101.   11.    5.   53.    3.   29.  716.   41.    3.   34.]
Dedf before if:  5
Target Node:  Node1@4
Node XI:  Node3@3
	G2: 10.7656226175
	Dedf:  5
	Assoc:  0


[[ 105.    3.    6.    2.]
 [   0.    2.   12.   47.]
 [   8.   48.  654.   76.]
 [   3.   22.    9.    3.]]
1000.0
[ 105.    3.    6.    2.    2.   12.   47.    8.   48.  654.   76.    3.
   22.    9.    3.]
Dedf before if:  8
Target Node:  Node1@4
Node XI:  Node4@4
	G2: 750.162044501
	Dedf:  8
	Assoc:  1.0


[[   5.   16.   95.]
 [   4.   12.   45.]
 [  59.  370.  357.]
 [   3.   22.   12.]]
1

[[   9.  107.]
 [   2.   59.]
 [  36.  750.]
 [   3.   34.]]
1000.0
[   9.  107.    2.   59.   36.  750.    3.   34.]
Dedf before if:  3
Target Node:  Node1@4
Node XI:  Node36@2
	G2: 2.97173053184
	Dedf:  3
	Assoc:  0


[[  27.   89.]
 [  10.   51.]
 [ 150.  636.]
 [   5.   32.]]
1000.0
[  27.   89.   10.   51.  150.  636.    5.   32.]
Dedf before if:  3
Target Node:  Node1@4
Node XI:  Node37@2
	G2: 2.35152821066
	Dedf:  3
	Assoc:  0


Universe actual size:
35
CPC actual size:
1
Target Node:  Node1@4
Node XI:  Node4@4
	G2: 5.22916090076
	Dedf:  1
	Assoc:  0.55577509678


Target Node:  Node1@4
Node XI:  Node5@3
	G2: 2.50016572929
	Dedf:  1
	Assoc:  0


Target Node:  Node1@4
Node XI:  Node7@3
	G2: 3.01606307868
	Dedf:  1
	Assoc:  0


Target Node:  Node1@4
Node XI:  Node8@3
	G2: 2.76113369926
	Dedf:  1
	Assoc:  0


Target Node:  Node1@4
Node XI:  Node10@3
	G2: 2.70673117724
	Dedf:  1
	Assoc:  0


Target Node:  Node1@4
Node XI:  Node12@3
	G2: 2.92453201309
	Dedf:  1
	Assoc:  0


Target Nod

	G2: 1085.19550257
	Dedf:  5
	Assoc:  1.0


[[ 108.    1.    7.    0.]
 [   0.   50.   11.    0.]
 [   1.    2.  782.    1.]
 [   3.    0.   10.   24.]]
1000.0
[ 108.    1.    7.   50.   11.    1.    2.  782.    1.    3.   10.   24.]
Dedf before if:  5
Target Node:  Node1@4
Node XI:  Node22@4
	G2: 1085.19550257
	Dedf:  5
	Assoc:  1.0


[[ 108.    1.    7.    0.]
 [   0.   50.   11.    0.]
 [   1.    2.  782.    1.]
 [   3.    0.   10.   24.]]
1000.0
[ 108.    1.    7.   50.   11.    1.    2.  782.    1.    3.   10.   24.]
Dedf before if:  5
Target Node:  Node1@4
Node XI:  Node22@4
	G2: 1085.19550257
	Dedf:  5
	Assoc:  1.0


[[ 108.    1.    7.    0.]
 [   0.   50.   11.    0.]
 [   1.    2.  782.    1.]
 [   3.    0.   10.   24.]]
1000.0
[ 108.    1.    7.   50.   11.    1.    2.  782.    1.    3.   10.   24.]
Dedf before if:  5
Target Node:  Node1@4
Node XI:  Node22@4
	G2: 1085.19550257
	Dedf:  5
	Assoc:  1.0


Target Node is:  Node1@4
PC set is :
FiO2@2
FiO2@2
FiO2@2


In [25]:
X_graph = pd.read_csv('./Alarm1_graph.txt',delimiter='  ',header=None)

/home/a20114261/anaconda3/envs/GeneInteractionsBN/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [272]:
X_graph = X_graph.transpose()

In [169]:
def related_nodes(X_graph,X,ind):
    print("Acoording to the true graph, the PC set for ",X.keys()[ind],"is:")
    for i in range(0,len(X_graph[ind])):
        if (X_graph[ind][i]==1):
            print(X.keys()[i])
    X_graph = X_graph.transpose()
    for i in range(0,len(X_graph[ind])):
        if (X_graph[ind][i]==1):
            print(X.keys()[i])
    X_graph = X_graph.transpose()

In [171]:
for i in range(0,len(X_graph.keys())):    
    print("====================================================")
    related_nodes(X_graph,X,i)

Acoording to the true graph, the PC set for  Node1@4 is:
Node22@4
Node29@3
Acoording to the true graph, the PC set for  Node2@4 is:
Node23@4
Node27@2
Node29@3
Acoording to the true graph, the PC set for  Node3@3 is:
Node30@2
Acoording to the true graph, the PC set for  Node4@4 is:
Node15@3
Node22@4
Acoording to the true graph, the PC set for  Node5@3 is:
Node6@2
Node13@3
Acoording to the true graph, the PC set for  Node6@2 is:
Node5@3
Acoording to the true graph, the PC set for  Node7@3 is:
Node9@2
Node13@3
Acoording to the true graph, the PC set for  Node8@3 is:
Node9@2
Node13@3
Acoording to the true graph, the PC set for  Node9@2 is:
Node7@3
Node8@3
Acoording to the true graph, the PC set for  Node10@3 is:
Node12@3
Node16@3
Acoording to the true graph, the PC set for  Node11@2 is:
Node36@2
Acoording to the true graph, the PC set for  Node12@3 is:
Node13@3
Node34@3
Node10@3
Acoording to the true graph, the PC set for  Node13@3 is:
Node14@2
Node5@3
Node7@3
Node8@3
Node12@3
Acoording to

In [167]:
related_nodes(X_graph,X,0)

Acoording to the true graph, the PC set for  Node1@4 is:
Node22@4 index:  22
Node29@3 index:  29


In [248]:
chi2.cdf(1,1)

0.68268949213708585

In [150]:
array_sum = np.array([[1,2],[3,4]])

In [151]:
np.sum(array_sum)

10

In [192]:
#Results for Node1 should { 2, 22, 23, 25 } 